In [1]:
from mqtt import Light_Switch
from IPython.display import display
import ipywidgets as widgets
import time, threading
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mc
import datetime
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import glob
import base64
from IPython.display import HTML


In [2]:
%matplotlib widget


In [3]:
class Controller:
    def __init__(self):
        self.mqtt = MQTT_connection("192.168.1.3")
        self.button = widgets.Button(description = "Switch Light Status")
        self.light_state = 0
        self.temperatures = []
        self.datetimes = []
        self.plot_data ={"time":range(len(self.temperatures)),"temperature":self.temperatures}        
        self.button.on_click(self.switch_lights)
        
        self.toggle = widgets.ToggleButton(description = "Switch Lights", value = bool(self.mqtt.light_state), 
                                            button_style = "info")
        self.toggle.observe(self.switch_lights, "value")
        display(self.toggle)
        
    def start_display(self):
        self.update_thread = threading.Thread(target = self.update_temperatures, args = ())
        self.update_thread.start()
   
    def switch_lights(self, x):
        #x passed because of how the ipywidgets button passes stuff to the connected function
        self.mqtt.switch_lights()
               
    
    def update_temperatures(self):
        
        fig, ax = plt.subplots()
        ax.autoscale(enable = True, axis = "both", tight = True)
        ax.set_xlabel("Time")
        ax.set_ylabel("Temperature (C)")
        ax.set_title(f"OptoWorld Live State")
        light_indicator = plt.Circle([0.8,1], radius = 0.01)
        ax.add_artist(light_indicator)
        line, = ax.plot([],[])
        plt.show()
        self.alive = True
        counter = 0
        while self.alive:
            self.temperatures.append(self.mqtt.q.get())
            self.mqtt.q.task_done()
            self.datetimes.append(datetime.datetime.now())
            
            
            counter +=1

            x_data = range(len(self.temperatures))
            line.set_xdata(self.datetimes)
            line.set_ydata(self.temperatures)
            light_indicator.center = [self.datetimes[-1] - datetime.timedelta(seconds = 30), 25]
#                 self.ax.relim()
#                 self.ax.autoscale_view(tight = True)
            ax.set_xlim([self.datetimes[0],self.datetimes[-1]+datetime.timedelta(seconds = 5)])
            ax.set_ylim(ymin = np.min(self.temperatures) -1, ymax = np.max(self.temperatures)+1)
            fig.canvas.draw()
            time.sleep(0.5)
                                        
            
            
            
            

In [ ]:
c = Controller()

In [ ]:
c.start_display()

In [4]:
def map_values(x, in_min, in_max, out_min, out_max):
    return (x-in_min) * (out_max - out_min) / (in_max - in_min) + out_min

def rle(inarray):
        """ run length encoding. Partial credit to R rle function. 
            Multi datatype arrays catered for including non Numpy
            returns: tuple (runlengths, startpositions, values) """
        ia = np.asarray(inarray)                  # force numpy
        n = len(ia)
        if n == 0: 
            return (None, None, None)
        else:
            y = np.array(ia[1:] != ia[:-1])     # pairwise unequal (string safe)
            i = np.append(np.where(y), n - 1)   # must include last element posi
            z = np.diff(np.append(-1, i))       # run lengths
            p = np.cumsum(np.append(0, z))[:-1] # positions
            return(z, p, ia[i])

In [5]:


widgets.interact_manual.opts["manual_name"] = "Update Plot"
@widgets.interact_manual
def plot_temperatures(file_to_plot = glob.glob("/mnt/NAS/optoworld_logs/*.txt"), 
                      time_range = widgets.IntRangeSlider(description = "range to plot", value = [0,100], min = 0, max = 100)):
    plt.close("all")
    fig1, axes = plt.subplots(figsize = (10,5))
    file = pd.read_csv(file_to_plot, sep = "\t")
    time_range = map_values(np.array([time_range[0], time_range[1]]), 0, 100, 0, len(file))
    lower, upper = time_range.astype(np.int32)
    file = file.loc[lower:upper]
    
    min_temp = file["temperature"].min()
    max_temp = file["temperature"].max()
    lengths, starts, values =rle(file["lightstate"])
    for length, start, value in zip(lengths, starts, values):
        if value == 1:
            print(length, start)
            x = mdates.date2num(pd.to_datetime(file["time"].iloc[start]))
            y = min_temp -1
            try:
                l = mdates.date2num(pd.to_datetime(file["time"].iloc[start+length])) - x
            except:
                l = mdates.date2num(pd.to_datetime(file["time"].iloc[-1])) - x
            h = max_temp - min_temp + 2
            rect = plt.Rectangle((x,y), l, h, facecolor = "lightblue", alpha = 0.3)
            axes.add_artist(rect)    
    axes.plot(pd.to_datetime(file["time"]), file["temperature"], c = "gray", alpha = 0.5, lw = 0.5)
    norm = mc.Normalize(vmin = 20, vmax = 30)
    axes.scatter(pd.to_datetime(file["time"]), file["temperature"], c = norm(file["temperature"].values), cmap = "RdBu_r", s = 1)
    fig1.autofmt_xdate()
    

interactive(children=(Dropdown(description='file_to_plot', options=('/mnt/NAS/optoworld_logs/20200226_optoworl…

In [6]:
def create_download_link(f, title = "Download CSV file", filename = "data.csv"):  
    filename = f
    title = f"Download file:  {filename}"
    filename = f.split("/")[-1]
    csv = open(f,'r').read()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


@widgets.interact
def download(filename = glob.glob("/mnt/NAS/optoworld_logs/*.txt")):
    display(create_download_link(filename))

interactive(children=(Dropdown(description='filename', options=('/mnt/NAS/optoworld_logs/20200226_optoworld_te…

In [10]:
def update_plot(event):
    ax.clear()
    file = pd.read_csv(file_dropdown.value, sep = "\t")
#     time_range = map_values(np.array([time_range[0], time_range[1]]), 0, 100, 0, len(file))
    time_range = np.array([0, len(file)])
    lower, upper = time_range.astype(np.int32)
    file = file.loc[lower:upper]
    
    min_temp = file["temperature"].min()
    max_temp = file["temperature"].max()
    lengths, starts, values =rle(file["lightstate"])
    for length, start, value in zip(lengths, starts, values):
        if value == 1:
#             print(length, start)
            x = mdates.date2num(pd.to_datetime(file["time"].iloc[start]))
            y = min_temp -1
            try:
                l = mdates.date2num(pd.to_datetime(file["time"].iloc[start+length])) - x
            except:
                l = mdates.date2num(pd.to_datetime(file["time"].iloc[-1])) - x
            h = max_temp - min_temp + 2
            rect = plt.Rectangle((x,y), l, h, facecolor = "lightblue", alpha = 0.3)
            ax.add_artist(rect)    
    ax.plot(pd.to_datetime(file["time"]), file["temperature"], c = "gray", alpha = 0.5, lw = 0.5)
    norm = mc.Normalize(vmin = 20, vmax = 30)
    ax.scatter(pd.to_datetime(file["time"]), file["temperature"], c = norm(file["temperature"].values), cmap = "RdBu_r", s = 1)
    fig.autofmt_xdate()

def show_download_link(event):
    display(create_download_link(file_dropdown.value))
    
def switch_light(event):
    switch.switch()
    
    
switch = Light_Switch(broker_address = "192.168.1.3", client_name = "switch")


In [11]:
fig, ax = plt.subplots();

file_dropdown = widgets.Dropdown(options = glob.glob("/mnt/NAS/optoworld_logs/*.txt"), value = glob.glob("/mnt/NAS/optoworld_logs/*.txt")[-1])
run_button = widgets.Button(description = "Update Plot")
run_button.on_click(update_plot)
download_button = widgets.Button(description = "Make Download Link")
download_button.on_click(show_download_link)
toggle_button = widgets.Button(description = "Switch Light")
toggle_button.on_click(switch_light)
update_plot("bah")
hbox = widgets.HBox(children = [file_dropdown, run_button, download_button, toggle_button])
display(hbox)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
type(light_monitor.light_state)

int